### Antoine EDY
# Natural Language Processing (COMM061) - Coursework

Introduction:


Preparation of the notebook:

In [10]:
import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset

Definition of the pipeline:

In [11]:
pipeline = {
    "pre-processing": None,
    "text-encoding": None,
    "algorithm": None,
}

def print_pipeline():
    for key, value in pipeline.items():
        print(f"{key}: {value}")

print_pipeline()

def train(train_dataset, validation_dataset, pipeline):
    pass

def evaluate(test_dataset, pipeline):
    pass

pre-processing: None
text-encoding: None
algorithm: None


## 1. Data Visualization

In [19]:
dataset = load_dataset("surrey-nlp/PLOD-CW")

def preprocess(df):
    df = df.drop(columns=['pos_tags'])
    return df


train_dataset = preprocess(pd.DataFrame(dataset['train']))
test_dataset = preprocess(pd.DataFrame(dataset['test']))
val_dataset = preprocess(pd.DataFrame(dataset['validation']))

train_dataset.info()

TEXT2ID = {
    "B-0": 0,
    "B-AC": 1,
    "B-LF": 2,
    "I-LF": 3,
}


# Here the exploration to add at the end of the work.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tokens    1072 non-null   object
 1   ner_tags  1072 non-null   object
dtypes: object(2)
memory usage: 16.9+ KB


In [20]:
train_dataset.head()

,tokens,ner_tags
0,"[For, this, purpose, the, Gothenburg, Young, P...","[B-O, B-O, B-O, B-O, B-LF, I-LF, I-LF, I-LF, I..."
1,"[The, following, physiological, traits, were, ...","[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, I-LF..."
2,"[Minor, H, antigen, alloimmune, responses, rea...","[B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O,..."
3,"[EPI, =, Echo, planar, imaging, .]","[B-AC, B-O, B-LF, I-LF, I-LF, B-O]"
4,"[Furthermore, ,, eNOS, -, derived, NO, S, -, n...","[B-O, B-O, B-AC, B-O, B-O, B-AC, B-O, B-O, B-O..."


# 2. Experimentations

### 1. Pre-processing techniques
1-gram, n-gram

In [14]:
from preprocessing import Preprocessing

pipeline["pre-processing"] = "Nothing" # Nothing, No-stop-words
print_pipeline()

preprocessing = Preprocessing(pipeline["pre-processing"])
preprocessing.fit(train_dataset)
train_dataset = preprocessing.transform(train_dataset)
val_dataset = preprocessing.transform(val_dataset)
test_dataset = preprocessing.transform(test_dataset)

pre-processing: Nothing
text-encoding: None
algorithm: None


### 2. Text encoding

In [15]:
from vectorization import Vectorization

pipeline["text-encoding"] = "Word2Vec" # Word2Vec
print_pipeline()

vectorization = Vectorization(pipeline["text-encoding"])
vectorization.fit(train_dataset)
w2v = vectorization.transform(train_dataset)

print(w2v)


pre-processing: Nothing
text-encoding: Word2Vec
algorithm: None
Word2Vec<vocab=10, vector_size=100, alpha=0.025>


In [16]:
from sklearn.decomposition import PCA

print(w2v.wv.key_to_index)
pca = PCA(n_components=2)

result = pca.fit_transform(X)

# create a scatter plot of the projection
plt.scatter(result[:, 0], result[:, 1])
words = list(w2v.wv.key_to_index)

for i, word in enumerate(words):
   plt.annotate(word, xy=(result[i, 0], result[i, 1]))

plt.show()

{'s': 0, 'n': 1, 'e': 2, 't': 3, 'g': 4, 'a': 5, '_': 6, 'r': 7, 'k': 8, 'o': 9}


NameError: name 'X' is not defined

### 3. NLP algorithms

Conditional Random Fields, RNNs, Transformers

### 4. Loss functions and Optimisers

# 3. Testing

# 4. Best model

# 5. Final evaluation